In [1]:
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque

import matplotlib.finance as finance
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from yahoo_finance import Share
import datetime
from bn_class import *

In [2]:
day_len = 10    # numbers of days for every data

In [3]:
def get_stock(ticker, startdate, enddate):
    fh = finance.fetch_historical_yahoo(ticker, startdate, enddate)
    # a numpy record array with fields: date, open, high, low, close, volume, adj_close)
    r = mlab.csv2rec(fh)
    fh.close()
    r.sort()
    print 'the length of data:', len(r.close)
    get_stock_data = []
    for i in xrange(0, len(r.close)-1):
        if (r.volume[i] != 0):
            get_stock_data.append(r.close[i].tolist())
    print 'after removing the datas with zero volume, the length of data:', len(get_stock_data)
    return get_stock_data

ticker = '2330.TW'

train = get_stock(ticker, datetime.date(2015, 1, 1), datetime.date(2015, 12, 31))
test = get_stock(ticker, datetime.date(2016, 1, 1), datetime.date.today())
max_ylim = max(max(train), max(test))
min_ylim = min(min(train), min(test))

the length of data: 261
after removing the datas with zero volume, the length of data: 242
the length of data: 160
after removing the datas with zero volume, the length of data: 146


In [7]:
'''
def save_pic(data, filename):
    for i in xrange (0, len(data)-day_len):
        fig, ax = plt.subplots(nrows=1, ncols=1)
        fig.set_size_inches(1, 1)
        ax.plot([i, i+1, i+2, i+3, i+4, i+5, i+6, i+7, i+8, i+9], [data[i], data[i+1], data[i+2], data[i+3], data[i+4], data[i+5], data[i+6], data[i+7], data[i+8], data[i+9]])
        ax.set_ylim([min_ylim, max_ylim])
        plt.axis('off')
        fig.savefig('/home/carine/Desktop/tmp/'+filename+'/'+filename+'_'+str(i)+'.png', dpi=80)
        fig.clear()
        plt.close(fig)

save_pic(train, "train")
save_pic(test, "test")
'''

In [13]:

def get_image(file_dir):
    img = mpimg.imread(file_dir)
    return img

image = []
for i in xrange(0, len(train)-day_len):
    file_dir = "/home/carine/Desktop/tmp/train/train_" + str(i) + ".png"
    image.append(get_image(file_dir))
my_train = np.asarray(image)

image = []
for i in xrange(0, len(test)-day_len):
    file_dir = "/home/carine/Desktop/tmp/test/test_" + str(i) + ".png"
    image.append(get_image(file_dir))
my_test = np.asarray(image)


In [14]:
my_stock_train = np.zeros((len(train)-day_len, day_len), dtype=np.float)
my_stock_test = np.zeros((len(test)-day_len, day_len), dtype=np.float)
for i in xrange(0, len(my_stock_train)):
    for j in xrange(0, day_len):
        my_stock_train[i,j] = train[i+j]

for i in xrange(0, len(my_stock_test)):
    for j in xrange(0, day_len):
        my_stock_test[i,j] = test[i+j]

In [15]:
class TWStock():
    def __init__(self, image_data, stock_price):
        self.image_data = image_data
        self.stock_price = stock_price
        self.stock_index = 0
    
    def render(self):
        return 
    
    def reset(self):
        self.stock_index = 0
        return self.image_data[self.stock_index]
    

    def step(self, action): 
        self.stock_index += 1
        action_reward = self.stock_price[self.stock_index][day_len-1] - self.stock_price[self.stock_index][day_len-2] 
        #action_reward = self.stock_price[self.stock_index+9] - self.stock_price[self.stock_index+8]
        if (action == 0):
            action_reward = 0
        if (action == 2):
            action_reward = -1 * action_reward

        stock_done = False
        if self.stock_index >= len(self.image_data)-1:
            stock_done = True
        else:
            stock_done = False
        return self.image_data[self.stock_index], action_reward, stock_done, 0

In [16]:
# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch

In [21]:
class DQN():
    # DQN Agent
    def __init__(self, env):
        # init experience replay
        self.replay_buffer = deque()

        # init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
    
        #self.state_dim = env.observation_space.shape[0]
        #self.action_dim = env.action_space.n
    
        self.state_dim = [1, 80, 80, 4]
        self.action_dim = 3


        self.create_Q_network()
        self.create_training_method()


        #g_record = tf.Graph()
        #self.g_session = tf.InteractiveSession(graph=g_record)
        self.t_session = tf.InteractiveSession()

        #with g_record.as_default():
        #self.R = tf.placeholder("float", shape = None)
        #R_summ = tf.scalar_summary(tags ="reward", values = self.R)

        #self.merged_summ = tf.merge_all_summaries()
        
        #self.writer = tf.train.SummaryWriter('/home/carine/Desktop/stock_DQN/eventlog',graph=self.t_session.graph)

        self.t_session.run(tf.initialize_all_variables())

    #def get_summ(self):
        #return self.t_session, self.merged_summ, self.R, self.writer

    def create_Q_network(self):
        # input layer
        self.state_input = tf.placeholder(tf.float32,[None ,80, 80, 4])

        
        # network weights
        W_conv1 = self.weight_variable([8, 8, 4, 32])
        b_conv1 = self.bias_variable([32])
        
        W_conv2 = self.weight_variable([4, 4, 32, 64])
        b_conv2 = self.bias_variable([64])

        W_conv3 = self.weight_variable([3, 3, 64, 64])
        b_conv3 = self.bias_variable([64])
        
        
        W_fc1 = self.weight_variable([1600, 512])
        b_fc1 = self.bias_variable([512])
        
        W_fc2 = self.weight_variable([512, self.action_dim])
        b_fc2 = self.bias_variable([self.action_dim])
        
        # hidden layers
        h_conv1 = tf.nn.relu(self.conv2d(self.state_input, W_conv1, 4) + b_conv1)
        h_pool1 = self.max_pool_2x2(h_conv1)
        h_conv2 = tf.nn.relu(self.conv2d(h_pool1, W_conv2, 2) + b_conv2)
        h_conv3 = tf.nn.relu(self.conv2d(h_conv2, W_conv3, 1) + b_conv3)
        h_conv3_flat = tf.reshape(h_conv3, [-1, 1600])
        h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)
        # Q Value layer
        self.Q_value = tf.matmul(h_fc1, W_fc2) + b_fc2
        
    def create_training_method(self):
        self.action_input = tf.placeholder(tf.float32,[None,self.action_dim])
        # one hot presentation
        self.y_input = tf.placeholder(tf.float32,[None])
        Q_action = tf.reduce_sum(tf.mul(self.Q_value,self.action_input),reduction_indices = 1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)

    def perceive(self,state,action,reward,next_state,done):
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))

        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()

    def train_Q_network(self):
        self.time_step += 1

        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]

        # Step 2: calculate y
        y_batch = []
        Q_value_batch = self.Q_value.eval(feed_dict={self.state_input:next_state_batch})

        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if done:
                y_batch.append(reward_batch[i])
            else :
                y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))

        self.optimizer.run(feed_dict={
          self.y_input: y_batch,
          self.action_input: action_batch,
          self.state_input: state_batch
          })
        

    def egreedy_action(self,state):
        Q_value = self.Q_value.eval(feed_dict = {
          self.state_input:[state]})[0]
        if random.random() <= self.epsilon:
            return random.randint(0,self.action_dim - 1)
        else:
            return np.argmax(Q_value)

        self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON)/10000


    def action(self,state):
        return np.argmax(self.Q_value.eval(feed_dict = {
          self.state_input:[state]})[0])


    def weight_variable(self,shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(self,shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(self, x, W, stride):
        return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

    def max_pool_2x2(self, x):
        return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

In [22]:
# Hyper Parameters
ENV_NAME = 'stock_2330-v0' 
EPISODE = 2000 #10000 # Episode limitation
STEP = 300 #1000   #300 # Step limitation in an episode
TEST = 10 #10 # The number of experiment test every 100 episode

In [25]:
def main():

    # initialize OpenAI Gym env and dqn agent
    #env = gym.make(ENV_NAME)
    env = TWStock(my_train,my_stock_train) 
    agent = DQN(env)
    #sess,merged,R,writer = agent.get_summ()

    print 'Start!'
    for episode in xrange(EPISODE):
    
        # initialize task
        state = env.reset()

        # Train
        for step in xrange(STEP):
            action = agent.egreedy_action(state) # e-greedy action for trai

            next_state,reward,done,_ = env.step(action)

            # Define reward for agent
            reward_agent = -1 if done else 0.1
            agent.perceive(state,action,reward,next_state,done)
            state = next_state
            if done:
                break
    

        # Test every 100 episodes
        if episode % 100 == 0:
            #env_test = TWStock(my_test, my_stock_test)
            total_reward = 0

            for i in xrange(TEST):
                state = env.reset()

                for j in xrange(STEP):
                    env.render()
                    action = agent.action(state)   # direct action for test
                    state,reward,done,_ = env.step(action)
                    total_reward += reward
                    if done:
                        break

            ave_reward = total_reward/TEST
            print 'episode: ',episode,'Evaluation Average Reward:',ave_reward
            #record = sess.run(merged, feed_dict={R:ave_reward})
            #writer.add_summary(record, global_step = episode)
            #writer.flush()
            #if ave_reward >= 200:
            #    print 'Done!' 
            #    break

In [ ]:
if __name__ == '__main__':
    main()

Start!
episode:  0 Evaluation Average Reward: -5.0
episode:  100 Evaluation Average Reward: 5.0
episode:  200 Evaluation Average Reward: -5.0
episode:  300 Evaluation Average Reward: 5.0
episode:  400 Evaluation Average Reward: 5.0
episode:  500 Evaluation Average Reward: 5.0
